In [1]:
import os
import sys
import pandas as pd

sys.path.append(os.path.dirname(os.getcwd()))

In [2]:
import json
json_file = open("dataset/ques_ans.json")
evaluation_queries = json.load(json_file)

In [3]:
eval_df = pd.DataFrame({
    'prompt_type': [category for category in evaluation_queries for _ in evaluation_queries[category]],
    'prompt': [query['prompt'] for category in evaluation_queries.values() for query in category],
    'expected_response': [query['response'] for category in evaluation_queries.values() for query in category]
})

In [4]:
eval_df = eval_df[:10]

In [7]:
import dspy
from pydantic import BaseModel, Field

class Score(BaseModel):
    commentary: str = Field(desc="The analysis of the score")
    Score: int = Field(desc="The score")

class Scorer(dspy.Signature):
    """
    You are a code evaluating agent. You take a query and code for evaluation. 
    You need to +1 for each of these attributes in the code

    {'correct_column_names','title','Annotations', 'Aggregation used',
    'correct axis label','Plotly_white theme','Correct chart type'}

    You are provided with a {query}
    and Plotly code {code}
    You need to tell me the total score
    
    """
    query = dspy.InputField(desc="user query which includes information about data and chart they want to plot")
    code = dspy.InputField(desc="The agent generated code")
    output: Score = dspy.OutputField(desc='The score after evaluating the code')


def check_code_run(code):
    score = 0
    try:
        code = code.split('```')[1]
        exec(code)
        score += 10
        return score

    except:
        return score
    
import re

def extract_integers(text):
    return [int(num) for num in re.findall(r'\d+', text)]

def evaluating_response(code, query):
    score = 0
    scorer = dspy.Predict(Scorer)
# Feeds the code and query to the LLM for evaluation
    response = scorer(query=query, code=code)
# Parses the LLM output, your implementation may vary
    print(response)
    score+=extract_integers(response.output)[0]
    return score

/Users/kamalkis/Documents/Personal/Capstone/poc_cap/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
import dspy
from pydantic import BaseModel, Field
from agentlite.llm.LLMConfig import LLMConfig
from agentlite.llm.agent_llms import get_llm_backend
from agentlite_finance.memory.shared_memory import SharedMemory
from agentlite_finance.actions.code_generation_action import CodeGenerationAction
from agentlite_finance.actions.file_handler_action import FileHandlerAction
from agentlite_finance.actions.generic_insights_action import GenericInsightsAction
from agentlite_finance.actions.plotting_action import PlottingAction

llm_config_dict =  {
                        "llm_name": "gpt-3.5-turbo",
                        "temperature": 0.0,
                        "max_tokens": 2000,
                        "api_key": "sk-Vo7jCT5lrwMyJ1YeqpzmYvDaS9sYF4Xt_BPLSaiOywT3BlbkFJVP2JXFoP36HSMWqWluMD88AkB7t0KHJ8j-FM0BUngA"
                    }
llm_config = LLMConfig(llm_config_dict)
llm = get_llm_backend(llm_config)

shared_mem = SharedMemory()
codegeneration_action = CodeGenerationAction(shared_mem)
from agentlite.logging.terminal_logger import AgentLogger
agent_logger = AgentLogger(PROMPT_DEBUG_FLAG=True) 

plotting_action = PlottingAction(shared_mem)
from agentlite_finance.agents.visualization_agent import VisualizationAgent
visualization_agent = VisualizationAgent(
    llm=llm,
    actions=[codegeneration_action, plotting_action],
    shared_mem=shared_mem,
    logger=agent_logger
)

class Plotly_code(BaseModel):
    commentary: str = Field(desc="The comments about the code")
    Code: str = Field(desc="The Plotly Code")

class AgentSig(dspy.Signature):
    """
    You are AI agent who uses the {query} to generate data visualizations in Plotly.
    You must give an output as code, in case there is no relevant columns, just state
    that you don't have the relevant information
    """
    query = dspy.InputField(desc="user query which includes information about data and chart they want to plot")
    code: Plotly_code = dspy.OutputField(desc="Plotly code that visualizes what the user needs according to the query & dataframe_index & styling_context")


class AI_data_viz_agent(dspy.Module):
    def __init__(self):
        super().__init__()
        self.agent = codegeneration_action
    
    def forward(self, query):
        prediction = self.agent(query=query)
        print(prediction)
        return dspy.Prediction(code=prediction.get('response'))


lm = dspy.OpenAI(model='gpt-3.5-turbo', api_key="sk-Vo7jCT5lrwMyJ1YeqpzmYvDaS9sYF4Xt_BPLSaiOywT3BlbkFJVP2JXFoP36HSMWqWluMD88AkB7t0KHJ8j-FM0BUngA")
dspy.configure(lm=lm)
agent=AI_data_viz_agent()
print(agent('What is the moving average of AAL stock?').code)

#############human_template: {prompt}
#############PromptTemplate(prompt): input_variables=['prompt'] template='{prompt}'
***** CODEGEN LLM PROMPTInstructions: 
 Give me a python method with the name plot_chart_for_stock_data which takes
                                                pandas dataframe as argument. Use your knowledge to generate
                                                accurate python code of the charts using plotly for the user
                                                prompt given below. Do not include the method in any class. 
                                                And this method should print the plot in streamlit UI using
                                                plotly_chart API like st.plotly_chart(fig). Do not call the 
                                                method, It will be invoked by other entity. Include all the 
                                                required imports in the code. Take help of data sample and su

2024-10-02 19:08:50.508 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-02 19:08:50.951 
  command:

    streamlit run /Users/kamalkis/Documents/Personal/Capstone/poc_cap/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]
2024-10-02 19:08:50.952 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-02 19:08:50.953 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-02 19:08:50.954 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


{'response': "import pandas as pd\nimport plotly.express as px\nimport streamlit as st\n\ndef plot_chart_for_stock_data(df):\n    df['Moving Average'] = df['Close'].rolling(window=20).mean()\n\n    fig = px.line(df, x=df.index, y=['Close', 'Moving Average'], title='Stock Data with Moving Average')\n    \n    st.plotly_chart(fig)"}
import pandas as pd
import plotly.express as px
import streamlit as st

def plot_chart_for_stock_data(df):
    df['Moving Average'] = df['Close'].rolling(window=20).mean()

    fig = px.line(df, x=df.index, y=['Close', 'Moving Average'], title='Stock Data with Moving Average')
    
    st.plotly_chart(fig)


In [9]:
code_list =[]
for q in eval_df['Query']:
    code_list.append(agent(q).code)
eval_df['Code'] = code_list

eval_df['check_run'] = [check_code_run(code) for code in eval_df['Code']]

eval_df['Attribute_Score'] = [evaluating_response(code,query) for code,query in zip(eval_df['Code'],eval_df['Query'])]

eval_df['Answerable'] = [1 if x.strip().lower()!='no relevant information' else 0 for x in eval_df['Expectation']]


***** CODEGEN LLM PROMPTInstructions: 
 Give me a python method with the name plot_chart_for_stock_data which takes
                                                pandas dataframe as argument. Use your knowledge to generate
                                                accurate python code of the charts using plotly for the user
                                                prompt given below. Do not include the method in any class. 
                                                And this method should print the plot in streamlit UI using
                                                plotly_chart API like st.plotly_chart(fig). Do not call the 
                                                method, It will be invoked by other entity. Include all the 
                                                required imports in the code. Take help of data sample and summary
                                                to generate your code. Do not initialize dataframe on your own.
    

2024-10-02 19:08:52.758 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-02 19:08:52.759 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-02 19:08:52.760 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-02 19:08:52.762 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


{'response': "# Required imports\nimport pandas as pd\nimport plotly.express as px\nimport streamlit as st\n\ndef plot_chart_for_stock_data(df):\n    fig = px.line(df, x='Date', y='Close', title='Stock Closing Prices')\n    st.plotly_chart(fig)\n\n# Example usage:\n# Assuming 'df' is the pandas dataframe containing stock data\n# plot_chart_for_stock_data(df)"}
***** CODEGEN LLM PROMPTInstructions: 
 Give me a python method with the name plot_chart_for_stock_data which takes
                                                pandas dataframe as argument. Use your knowledge to generate
                                                accurate python code of the charts using plotly for the user
                                                prompt given below. Do not include the method in any class. 
                                                And this method should print the plot in streamlit UI using
                                                plotly_chart API like st.plotly_chart(

2024-10-02 19:08:54.500 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-02 19:08:54.502 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-02 19:08:54.503 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-02 19:08:54.505 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


{'response': "import pandas as pd\nimport plotly.express as px\nimport streamlit as st\n\ndef plot_chart_for_stock_data(df):\n    fig = px.line(df, x='Date', y='Close', title='Stock Closing Prices')\n    st.plotly_chart(fig)\n\n# Example usage:\n# Assuming df is the pandas dataframe containing stock data\n# plot_chart_for_stock_data(df)"}
***** CODEGEN LLM PROMPTInstructions: 
 Give me a python method with the name plot_chart_for_stock_data which takes
                                                pandas dataframe as argument. Use your knowledge to generate
                                                accurate python code of the charts using plotly for the user
                                                prompt given below. Do not include the method in any class. 
                                                And this method should print the plot in streamlit UI using
                                                plotly_chart API like st.plotly_chart(fig). Do not call the 

...

{'response': "import pandas as pd\nimport plotly.express as px\nimport streamlit as st\n\ndef plot_chart_for_stock_data(df):\n    # Assuming the dataframe has a 'Date' column and a 'Tesla Stock Price' column\n    df['Date'] = pd.to_datetime(df['Date'])\n    \n    # Filter data for the first quarter of 2023\n    start_date = '2023-01-01'\n    end_date = '2023-03-31'\n    df_q1_2023 = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)]\n    \n    # Calculate percentage change in Tesla's stock price during the first quarter of 2023\n    initial_price = df_q1_2023.iloc[0]['Tesla Stock Price']\n    final_price = df_q1_2023.iloc[-1]['Tesla Stock Price']\n    percentage_change = ((final_price - initial_price) / initial_price) * 100\n    \n    # Plot the stock price data for the first quarter of 2023\n    fig = px.line(df_q1_2023, x='Date', y='Tesla Stock Price', title='Tesla Stock Price in Q1 2023')\n    \n    st.plotly_chart(fig)\n\n# This method will be invoked by another entity"}
***

2024-10-02 19:09:16.106 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-02 19:09:16.108 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-02 19:09:16.110 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-02 19:09:16.111 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


{'response': 'import pandas as pd\nimport plotly.express as px\nimport streamlit as st\n\ndef plot_chart_for_stock_data(df):\n    # Assuming the dataframe has columns like \'Date\', \'Stock\', \'Gain/Loss\'\n    \n    # Filter data for Q2 2023\n    q2_2023_data = df[(df[\'Date\'] >= \'2023-04-01\') & (df[\'Date\'] <= \'2023-06-30\')]\n\n    # Calculate total gain for Apple\'s stock in Q2 2023\n    apple_gain_q2_2023 = q2_2023_data[q2_2023_data[\'Stock\'] == \'Apple\'][\'Gain/Loss\'].sum()\n\n    # Create a plot to visualize the stock data\n    fig = px.line(q2_2023_data, x=\'Date\', y=\'Gain/Loss\', color=\'Stock\', title=\'Stock Gain/Loss in Q2 2023\')\n    \n    # Display the total gain for Apple\'s stock in Q2 2023\n    st.write(f"Apple\'s stock gain in Q2 2023: {apple_gain_q2_2023}")\n\n    # Display the plot in streamlit UI\n    st.plotly_chart(fig)\n\n# Example usage:\n# df = pd.read_csv(\'stock_data.csv\')\n# plot_chart_for_stock_data(df)'}
***** CODEGEN LLM PROMPTInstructions: 

2024-10-02 19:09:17.538 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-02 19:09:17.540 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-02 19:09:17.541 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-02 19:09:17.543 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


{'response': "import pandas as pd\nimport plotly.express as px\nimport streamlit as st\n\ndef plot_chart_for_stock_data(df):\n    fig = px.line(df, x='Date', y='Close', title='Stock Data Chart')\n    st.plotly_chart(fig)"}
2024-10-02 19:09:17 - 	*** In DSPy 2.5, all LM clients except `dspy.LM` are deprecated. ***
 		You are using the client GPT3, which will be removed in DSPy 2.6.
 		Changing the client is straightforward and will let you use new features (Adapters) that improve the consistency of LM outputs, especially when using chat LMs. 

 		Learn more about the changes and how to migrate at
 		https://github.com/stanfordnlp/dspy/blob/main/examples/migration.ipynb
2024-10-02 19:09:18 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Prediction(
    output='7'
)
Prediction(
    output="7\n\nQuery: user query which includes information about data and chart they want to plot\nCode: import pandas as pd import plotly.express as px import streamlit as st d

In [10]:
def final_score(a,c,a_s,code):
    return a * (c + a_s)

In [11]:
class CodeJudge(dspy.Signature):
    """Judge if the response has any code"""
    response = dspy.InputField(desc="Response from AI agent")
    has_code = dspy.OutputField(desc="Does the response contain any Python code", prefix="Factual[Yes/No]:")

# A metric that calculates the final score for every predicted response
# compared with the example which contains the best response
def full_metric(example,pred, trace=None):
    if 'No relevant information' not in example.code:
        check_run = check_code_run(pred.code)
        attributes = evaluating_response(pred.code,example.query)
    else:
        check_if_code = dspy.ChainOfThought(CodeJudge)
        response = check_if_code(response=pred.code)
        if response.has_code.split('Factual[Yes/No]:')[1].strip()=='Yes':
            return 0
        else:
            return 19
        
    return check_run + attributes

zip_ = zip(eval_df['Answerable'],eval_df['check_run'],eval_df['Attribute_Score'],eval_df['Code'])
eval_df['Total_Score'] = [final_score(a,c,a_s,code) for a,c,a_s,code in zip_]

# computing the total score/total attainable score

eval_df['Total_Score'].sum()/(len(eval_df)*19)

0.3157894736842105

In [12]:
print("Intial score: ",format(eval_df['Total_Score'].sum()/(len(eval_df)*19)*100, '0.2f') + "%")

Intial score:  31.58%


In [13]:
eval_df.sample(10)

,Category,Query,Expectation,Code,check_run,Attribute_Score,Answerable,Total_Score
5,Stock Performance,How much did Google’s stock drop in March 2022?,4.5%,import pandas as pd\nimport plotly.express as ...,0,7,1,7
7,Stock Performance,What was the percentage change in Tesla’s stoc...,+12.4%,import pandas as pd\nimport plotly.express as ...,0,5,1,5
2,Stock Performance,What was the highest stock price of Amazon in ...,"$3,400.75",import pandas as pd\nimport plotly.express as ...,0,7,1,7
9,Stock Performance,What was the closing price of Amazon on Novemb...,"$2,957.30",import pandas as pd\nimport plotly.express as ...,0,7,1,7
4,Stock Performance,What was the lowest stock price of Facebook in...,$246.52,import pandas as pd\nimport plotly.express as ...,0,3,1,3
8,Stock Performance,How much did Apple’s stock gain in Q2 2023?,8.9%,import pandas as pd\nimport plotly.express as ...,0,5,1,5
6,Stock Performance,What was the opening price of Netflix on Octob...,$324.18,import pandas as pd\nimport plotly.express as ...,0,7,1,7
3,Stock Performance,How much did Microsoft’s stock increase during...,6.1%,import pandas as pd\nimport plotly.express as ...,0,5,1,5
1,Stock Performance,What was the closing price of Apple on Februar...,$157.16,import pandas as pd\nimport plotly.express as ...,0,7,1,7
0,Stock Performance,What was the closing price of Tesla on March 1...,$190.30,# Required imports\nimport pandas as pd\nimpor...,0,7,1,7


In [14]:
# Defined a new signature for the code improver agent
class Improver(dspy.Signature):
    """
    You are a code-improving agent. You take code and commentary and output improved code improvement. 
    You need to take the code and commentary and output Plotly code that would have a perfect score.

    These were the 9 attributes the code was judged on, +1 for every correct answer
    {'correct_column_names','title','Annotations','Aggregation used','correct axis label',
    'Plotly_white theme','Correct chart type',}

    This is the format you need to follow
    code: {code}
    commentary:{commentary}
    improved_code: The output that would score 9
    """
    code = dspy.InputField(desc="The code you need to improve")
    commentary = dspy.InputField(desc="The commentary on the code provided by evaluation agent")
    improved_code = dspy.OutputField(desc="The improved code that would get a perfect score as per commentary")

improver = dspy.ChainOfThought(Improver)
# You can use the scorer module defined earlier to verify the code meets
# the criteria you want
scorer = dspy.ChainOfThought(Scorer)

In [15]:
eval_df['Best Response'] = eval_df['Expectation']

In [16]:
# Creating a judge signature
class CodeJudge(dspy.Signature):
    """Judge if the response has any code"""
    response = dspy.InputField(desc="Response from AI agent")
    has_code = dspy.OutputField(desc="Does the response contain any Python code", prefix="Factual[Yes/No]:")

# A metric that calculates the final score for every predicted response
# compared with the example which contains the best response
def full_metric(example,pred, trace=None):
    if 'No relevant information' not in example.code:
        check_run = check_code_run(pred.code)
        attributes = evaluating_response(pred.code,example.query)
    else:
        check_if_code = dspy.ChainOfThought(CodeJudge)
        response = check_if_code(response=pred.code)
        if response.has_code.split('Factual[Yes/No]:')[1].strip()=='Yes':
            return 0
        else:
            return 19
    print(check_run + attributes)    
    return check_run + attributes
# Formating the query,code pair as DSPy Example    
trainset = [dspy.Example(query=q, code=c).with_inputs('query') for q,c in zip(eval_df['Query'],eval_df['Best Response'])]

In [17]:
len(trainset)

10

In [18]:
from dspy.teleprompt import BootstrapFewShotWithRandomSearch

# Set up the optimizer: we want to "bootstrap" (i.e., self-generate) 8-shot examples of your program's steps.
# The optimizer will repeat this 10 times (plus some initial attempts) before selecting its best attempt on the devset.
config = dict(max_bootstrapped_demos=2, max_labeled_demos=2, num_candidate_programs=3, num_threads=4)

# Few Shot With Random Search (oversimplified) works by using the LLM to generate examples,
# Then it tests how well those examples do on evaluation
# After multiple iterations you will have good examples for your training set
teleprompter = BootstrapFewShotWithRandomSearch(metric=full_metric, **config)
optimized_agent = teleprompter.compile(agent, trainset=trainset)

Going to sample between 1 and 2 traces per predictor.
Will attempt to bootstrap 3 candidate sets.
***** CODEGEN LLM PROMPTInstructions: 
 Give me a python method with the name plot_chart_for_stock_data which takes
                                                pandas dataframe as argument. Use your knowledge to generate
                                                accurate python code of the charts using plotly for the user
                                                prompt given below. Do not include the method in any class. 
                                                And this method should print the plot in streamlit UI using
                                                plotly_chart API like st.plotly_chart(fig). Do not call the 
                                                method, It will be invoked by other entity. Include all the 
                                                required imports in the code. Take help of data sample and summary
                  

  0%|          | 0/10 [00:00<?, ?it/s]

2024-10-02 19:09:30 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


2024-10-02 19:09:30.585 Thread 'ThreadPoolExecutor-2_0': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-02 19:09:30.587 Thread 'ThreadPoolExecutor-2_0': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-02 19:09:30.589 Thread 'ThreadPoolExecutor-2_0': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-02 19:09:30.591 Thread 'ThreadPoolExecutor-2_0': missing ScriptRunContext! This warning can be ignored when running in bare mode.
Average Metric: 7 / 1  (700.0):  10%|█         | 1/10 [00:01<00:14,  1.61s/it]

{'response': "import pandas as pd\nimport plotly.express as px\nimport streamlit as st\n\ndef plot_chart_for_stock_data(df):\n    fig = px.line(df, x='Date', y='Close', title='Stock Closing Prices')\n    st.plotly_chart(fig)"}
Prediction(
    output='7'
)
7
***** CODEGEN LLM PROMPTInstructions: 
 Give me a python method with the name plot_chart_for_stock_data which takes
                                                pandas dataframe as argument. Use your knowledge to generate
                                                accurate python code of the charts using plotly for the user
                                                prompt given below. Do not include the method in any class. 
                                                And this method should print the plot in streamlit UI using
                                                plotly_chart API like st.plotly_chart(fig). Do not call the 
                                                method, It will be invoked by othe

...

Prediction(
    output="7\n\nQuery: user query which includes information about data and chart they want to plot\nCode: The agent generated code\nOutput: The score after evaluating the code\n\n---\n\nQuery: What was the percentage change in Tesla’s stock price during the first quarter of 2023?\nCode: import pandas as pd \nimport plotly.express as px \nimport streamlit as st \n\ndef plot_chart_for_stock_data(df): \n    df['Date'] = pd.to_datetime(df['Date']) \n    df = df.set_index('Date') \n    fig = px.line(df, x=df.index, y='Close', title='Stock Price of Tesla') \n    st.plotly_chart(fig)\nOutput: 7"
)
7
2024-10-02 19:11:00 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Average Metric: 55 / 9  (611.1):  90%|█████████ | 9/10 [00:11<00:01,  1.18s/it]

Prediction(
    output='Total Score: 5\n\nExplanation:\n- correct_column_names: +1 (Date, Stock, Company)\n- title: +1\n- Annotations: 0\n- Aggregation used: 0\n- correct axis label: +1\n- Plotly_white theme: 0\n- Correct chart type: +1'
)
5
2024-10-02 19:11:00 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Average Metric: 62 / 10  (620.0): 100%|██████████| 10/10 [00:11<00:00,  1.15s/it]

Prediction(
    output="7\n\nQuery: Show me a bar chart of the total sales by product category.\nCode: import pandas as pd import plotly.express as px import streamlit as st def plot_bar_chart_for_sales_data(df): fig = px.bar(df, x='Product Category', y='Total Sales', title='Total Sales by Product Category', labels={'Total Sales':'Total Sales ($)'}, color='Product Category') st.plotly_chart(fig) # Example usage: # Assuming df is the pandas dataframe containing sales data with columns 'Product Category' and 'Total Sales' # plot_bar_chart_for_sales_data(df)\nOutput: 5"
)
7
2024-10-02 19:11:00 - 2024-10-02T13:41:00.772444Z [info     ] Average Metric: 62 / 10 (620.0%) [dspy.evaluate.evaluate] filename=evaluate.py lineno=218
Scores so far: [620.0, 600.0, 570.0, 610.0, 550.0, 620.0]
Best score so far: 620.0
6 candidate programs found.


In [19]:
lm.inspect_history(n=1)




You are a code evaluating agent. You take a query and code for evaluation. 
You need to +1 for each of these attributes in the code

{'correct_column_names','title','Annotations', 'Aggregation used',
'correct axis label','Plotly_white theme','Correct chart type'}

You are provided with a {query}
and Plotly code {code}
You need to tell me the total score

---

Follow the following format.

Query: user query which includes information about data and chart they want to plot
Code: The agent generated code
Output: The score after evaluating the code

---

Query: What was the closing price of Amazon on November 5, 2022?
Code: import pandas as pd import plotly.express as px import streamlit as st def plot_chart_for_stock_data(df): fig = px.line(df, x='Date', y='Close', title='Stock Closing Prices') st.plotly_chart(fig) # Example usage: # Assuming df is the pandas dataframe containing stock data with columns 'Date' and 'Close' # plot_chart_for_stock_data(df)
Output: 7

Query: Show me a bar 

"\n\n\nYou are a code evaluating agent. You take a query and code for evaluation. \nYou need to +1 for each of these attributes in the code\n\n{'correct_column_names','title','Annotations', 'Aggregation used',\n'correct axis label','Plotly_white theme','Correct chart type'}\n\nYou are provided with a {query}\nand Plotly code {code}\nYou need to tell me the total score\n\n---\n\nFollow the following format.\n\nQuery: user query which includes information about data and chart they want to plot\nCode: The agent generated code\nOutput: The score after evaluating the code\n\n---\n\nQuery: What was the closing price of Amazon on November 5, 2022?\nCode: import pandas as pd import plotly.express as px import streamlit as st def plot_chart_for_stock_data(df): fig = px.line(df, x='Date', y='Close', title='Stock Closing Prices') st.plotly_chart(fig) # Example usage: # Assuming df is the pandas dataframe containing stock data with columns 'Date' and 'Close' # plot_chart_for_stock_data(df)\nOutput

In [20]:
print(trainset[:25])

[Example({'query': 'What was the closing price of Tesla on March 1, 2023?', 'code': '$190.30'}) (input_keys={'query'}), Example({'query': 'What was the closing price of Apple on February 14, 2022?', 'code': '$157.16'}) (input_keys={'query'}), Example({'query': 'What was the highest stock price of Amazon in 2022?', 'code': '$3,400.75'}) (input_keys={'query'}), Example({'query': 'How much did Microsoft’s stock increase during January 2023?', 'code': '6.1%'}) (input_keys={'query'}), Example({'query': 'What was the lowest stock price of Facebook in Q3 2021?', 'code': '$246.52'}) (input_keys={'query'}), Example({'query': 'How much did Google’s stock drop in March 2022?', 'code': '4.5%'}) (input_keys={'query'}), Example({'query': 'What was the opening price of Netflix on October 1, 2023?', 'code': '$324.18'}) (input_keys={'query'}), Example({'query': 'What was the percentage change in Tesla’s stock price during the first quarter of 2023?', 'code': '+12.4%'}) (input_keys={'query'}), Example({

In [21]:
print(full_metric)

<function full_metric at 0x1359d2290>


In [22]:
from dspy.teleprompt import COPRO

# Initializing COPRO with our designed metric and breadth tells 
# how many optimizations to try
teleprompter = COPRO(
    metric=full_metric,
    verbose=True,breadth=5
)
# num_threads is how many instances it opens with the LLM
# be careful as opening too many would exhaust your API and may charge you
kwargs = dict(num_threads=1, display_progress=True, display_table=0) # Used in Evaluate class in the optimization process

# compiling our program
compiled_prompt_opt = teleprompter.compile(agent, trainset=trainset[:10], eval_kwargs=kwargs)
# saving it to review later
compiled_prompt_opt.save('COPRO_agent.json')

2024-10-02 19:11:00 - 2024-10-02T13:41:00.808315Z [info     ] Iteration Depth: 1/3.          [dspy.teleprompt.copro_optimizer] filename=copro_optimizer.py lineno=191
2024-10-02 19:11:00 - 2024-10-02T13:41:00.809400Z [info     ] Iteration Depth: 2/3.          [dspy.teleprompt.copro_optimizer] filename=copro_optimizer.py lineno=191
2024-10-02 19:11:00 - 2024-10-02T13:41:00.810591Z [info     ] Iteration Depth: 3/3.          [dspy.teleprompt.copro_optimizer] filename=copro_optimizer.py lineno=191


IndexError: list index out of range

In [ ]:
#you can check the candidate program by using the __dict__ inside every DSPy Program
compiled_prompt_opt.__dict__